Initialize Google Earth Engine (GEE)

In [1]:
import os, ee, geemap

ee.Authenticate()
ee.Initialize()

# Setup map
Map = geemap.Map()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


Create Area of Interest (Dixie Fire California)

In [2]:
# Dixie Fire approximate bounding box (Butte/Tehama/Plumas counties, CA)
aoi = ee.Geometry.Polygon(
    [[[-121.8, 39.5], [-121.8, 40.3], [-120.7, 40.3], [-120.7, 39.5]]]
)
Map.centerObject(aoi, 8)
Map.addLayer(aoi, {}, "Dixie Fire AOI")
Map


Map(center=[39.90052064128846, -121.25000000000004], controls=(WidgetControl(options=['position', 'transparent…

Load and Explore Each Dataset

In [3]:
# MODIS Burned Area
modis = ee.ImageCollection("MODIS/006/MCD64A1").filterDate("2021-07-01", "2021-10-01").select("BurnDate")
modis_mosaic = modis.mosaic().clip(aoi)

Map.addLayer(modis_mosaic, {"min": 1, "max": 366, "palette": ["yellow", "red"]}, "MODIS Burned Area")

# VIIRS Active Fires
viirs = ee.ImageCollection("NASA/LANCE/NOAA20_VIIRS/C2").filterDate("2021-07-01", "2021-10-01").select("FRP")
viirs_mosaic = viirs.mosaic().clip(aoi)

Map.addLayer(viirs_mosaic, {"min": 0, "max": 500, "palette": ["blue", "orange", "red"]}, "VIIRS FRP")

# Sentinel-2 NDVI
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

s2 = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterDate("2021-06-01", "2021-07-01") \
    .filterBounds(aoi) \
    .map(maskS2clouds)

ndvi = s2.mean().normalizedDifference(["B8", "B4"]).rename("NDVI")
Map.addLayer(ndvi, {"min": 0, "max": 1, "palette": ["white", "green"]}, "Sentinel-2 NDVI (Pre-Fire)")

# ESA WorldCover
landcover = ee.ImageCollection("ESA/WorldCover/v100").first().clip(aoi)
Map.addLayer(landcover, {}, "ESA Land Cover")

# ERA5 Land Climate (Temperature)
era5 = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
    .filterDate("2021-07-01", "2021-07-31") \
    .select("temperature_2m")

era5_mean_temp = era5.mean().clip(aoi)
Map.addLayer(era5_mean_temp, {"min": 290, "max": 310, "palette": ["blue", "yellow", "red"]}, "ERA5 Mean Temp (July 2021)")

# CHIRPS Precipitation
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
    .filterDate("2021-06-01", "2021-07-01") \
    .select("precipitation")

chirps_total = chirps.sum().clip(aoi)
Map.addLayer(chirps_total, {"min": 0, "max": 200, "palette": ["white", "blue"]}, "CHIRPS Precipitation (June 2021)")

# SRTM Digital Elevation Model
srtm = ee.Image("USGS/SRTMGL1_003").clip(aoi)
Map.addLayer(srtm, {"min": 0, "max": 3000, "palette": ["white", "black"]}, "SRTM DEM")

Map

Map(center=[39.90052064128846, -121.25000000000004], controls=(WidgetControl(options=['position', 'transparent…

Save Raw Data

In [4]:
output_folder = "../data/raw"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
datasets = {
    "dixie_modis_burned_area": modis_mosaic,
    "dixie_viirs_frp": viirs_mosaic,
    "dixie_ndvi_prefire": ndvi,
    "dixie_esa_landcover": landcover,
    "dixie_era5_mean_temp": era5_mean_temp,
    "dixie_chirps_precip": chirps_total,
    "dixie_srtm_dem": srtm
}

for name, image in datasets.items():
    out_path = os.path.join(output_folder, f"{name}.tif")
    print(f"Exporting {name}...")
    geemap.ee_export_image(
        image.reproject(crs='EPSG:4326', scale=30),
        filename=out_path,
        scale=30 if name != "dixie_modis_burned_area" else 500,
        region=aoi,
        file_per_band=False
    )

Exporting dixie_modis_burned_area...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\John Waugh\Desktop\CS Projects\Wildfire-Lifecycle\data\raw\dixie_modis_burned_area.tif
Exporting dixie_viirs_frp...
Generating URL ...
An error occurred while downloading.
Expression evaluates to an image with no bands.
Exporting dixie_ndvi_prefire...
Generating URL ...
An error occurred while downloading.
Total request size (60734625 bytes) must be less than or equal to 50331648 bytes.
Exporting dixie_esa_landcover...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\John Waugh\Desktop\CS Projects\Wildfire-Lifecycle\data\raw\dixie_esa_landcover.tif
Exporting dixie_era5_mean_temp...
Generating URL ...
An error occurred while downloading.
Total request size (109322325 bytes) must be less than or equal to 50331648 bytes.
Exporting dixie_chirps_precip...
Generating URL ...
An error occurred while downloading.
Total request size (109322325 bytes) must be less than or equal to 50